## Starting the environment

In [1]:
from TableTennisEnvironmentV0.TableTennisEnvironment import TableTennisEnv
from utils.BotPluginMoveIt import BotPluginMoveIt
import pybullet as p
import numpy as np
import time
import joblib
from utils.physics_solvers import estimateInitVelocity, estimate_hitting_point, calculate_delta, had_double_bounce, had_no_bounce
import math
import torch
import torch.nn as nn
import torch.optim as optim
from delta_estimator_mlp.DeltaEstimatorMLP import DeltaEstimatorMLP
import random

pybullet build time: Nov 28 2023 23:51:11


Predicted Output: [-0.2558609 -1.5050874]


/home/n3r0/.local/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
env = TableTennisEnv(show_gui=False)

## MoveItPlugin

In [3]:
moveit_plugin = BotPluginMoveIt(environment = env)

[ERROR] [1718316324.314208101]: Group 'paddle_group' for end-effector 'paddle' cannot be its own parent
[ WARN] [1718316324.352050915]: The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
[ WARN] [1718316324.374067282]: The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
[ WARN] [1718316324.388459820]: The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.


In [20]:
xt = -0.4
yt = -0.5
zt = 1.7

moveit_plugin.group.set_named_target('ready_to_play_pose')
moveit_plugin.plan = moveit_plugin.group.plan()
moveit_plugin.execute_plan()
moveit_plugin.plan_motion(x = xt,
                          y = yt,## This goal position denotes the intercept point
                          z = zt - 1.4,
                          roll = 0,
                          pitch = 0,
                          yaw = np.pi/2)
plan1 = moveit_plugin.plan
plan1_trajectory = plan1[1].joint_trajectory.points

moveit_plugin.plan_motion(x = xt,
                          y = yt,## This goal position denotes the intercept point
                          z = zt - 1.4,
                          roll = 0,
                          pitch = 0,
                          yaw = -np.pi/2)
plan2 = moveit_plugin.plan
plan2_trajectory = plan2[1].joint_trajectory.points

print(len(plan1_trajectory), len(plan2_trajectory))

if len(plan1_trajectory)< len(plan2_trajectory):
    moveit_plugin.plan = plan1
else:
    moveit_plugin.plan = plan2
# print(moveit_plugin.plan)
# moveit_plugin.execute_plan() ## Reached the intercept point

## 2 acceptable orienations [0, 0, +- np.pi/2]
## PLAN MOTION for each of these orientations
## find the length of these orientations

53 74


In [5]:
counter = 0
for step in moveit_plugin.plan[1].joint_trajectory.points:
    joint_angles = step.positions
    counter += 1

    if counter > 1:
        for i, angle in enumerate(joint_angles):
            p.setJointMotorControl2(bodyIndex=moveit_plugin.env._robotic_arm,
                                    jointIndex=i,
                                    controlMode=p.POSITION_CONTROL,
                                    targetPosition=angle,
                                   )
            p.stepSimulation()
            time.sleep(1/240.0)
    
moveit_plugin.group.go(wait = True)

True

## Delta Estimator (MLP)

In [6]:
# Load the scaler
scaler = joblib.load('delta_estimator_mlp/scaler.pkl')

# Parameters
input_size = 9  # Number of input features
hidden_size = 32  # Number of neurons in the hidden layer
output_size = 2  # Number of output features (delta_y, delta_z)

# Create model and load trained weights
model = DeltaEstimatorMLP(input_size, hidden_size, output_size)
model.load_state_dict(torch.load('delta_estimator_mlp/delta_estimator_mlp.pth'))
model.eval()

# Sample inference code
def calculate_delta_v2(input_features):
    # Convert input features to tensor
    input_tensor = torch.tensor(input_features, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    
    # Perform inference
    with torch.no_grad():
        prediction = model(input_tensor)

    # print(prediction)
    # Inverse transform the prediction to get original values
    prediction_original = scaler.inverse_transform(prediction.numpy())
    
    return prediction_original[0]

# Example input features for inference
sample_input = np.array([
    0.5, 0.5, 0.5,  # ball_initial_pos_x, ball_initial_pos_y, ball_initial_pos_z
    1.0,  # distance_ball_robot
    0.1, 0.1, 0.1,  # estimated_initial_velocity_x, estimated_initial_velocity_y, estimated_initial_velocity_z
    0.2, 0.2  # estimated_hitting_point_y, estimated_hitting_point_z
])

# Perform inference and print the result
predicted_output = calculate_delta_v2(sample_input)
print("Predicted Output:", predicted_output)


Predicted Output: [-0.2558609 -1.5050874]


/home/n3r0/.local/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Paddle Goal Calculator

In [7]:
def find_goal_position():
    ball_position1 = ball_initial_position = p.getBasePositionAndOrientation(env._ball)[0]
    robot_initial_position = p.getLinkState(env._robotic_arm, 0)[0]      # can be collected from optitrack
    distance = robot_initial_position[0] - ball_initial_position[0]
    action = [0, 0, 0, 0, 0, 0]
    env.step(action)
    ball_position2 = p.getBasePositionAndOrientation(env._ball)[0]
    table_height = 1.0
    estimated_initial_velocity = estimateInitVelocity(ball_position1, ball_position2, 1/240)
    estimated_hitting_point, t1, t2 = estimate_hitting_point(ball_initial_position, robot_initial_position, estimated_initial_velocity, table_height)
    input_for_delta = [ball_initial_position[0], 
                       ball_initial_position[1], 
                       ball_initial_position[2], 
                       distance, 
                       estimated_initial_velocity[0], 
                       estimated_initial_velocity[1], 
                       estimated_initial_velocity[2], 
                       estimated_hitting_point[0], 
                       estimated_hitting_point[1]]
    delta = calculate_delta_v2(np.asarray(input_for_delta))
    return estimated_hitting_point+delta, estimated_initial_velocity

## Additional Info
Ball position range:
- x : [-2, -1]
- y : [-0.5, +0.5]
- z : [1.5, 2.5]

Ball velocity range:
- x : [4, 8]
- y : [-0.5, +0.5]
- z : [-3, +1]



## Collision Checker

In [8]:
def check_collision(body1, body2):
    contacts = p.getContactPoints(body1, body2)
    return len(contacts)>0

## Defining the Neural Net

In [9]:
class HittingVelocityPlanner(nn.Module):
    def __init__(self, features_len, vel_len):
        super(HittingVelocityPlanner, self).__init__()
        self.fc0 = nn.Linear(features_len, 2*features_len)
        self.fc1 = nn.Linear(2*features_len, 4*features_len)
        self.fc2 = nn.Linear(4*features_len, 8*features_len)
        self.fc3 = nn.Linear(8*features_len, 4*features_len)
        self.fc4 = nn.Linear(4*features_len, 2*features_len)
        self.fc5 = nn.Linear(2*features_len, vel_len)

    def forward(self, x):
        x = x0 = self.fc0(x)
        x = x1 = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        x = x1 + torch.relu(self.fc3(x))
        x = x0 + self.fc4(x)
        x = self.fc5(x)
        return x


batch_size = 1
features_len = 9
vel_len = 3

hitting_model = HittingVelocityPlanner(features_len, vel_len)
x = torch.randn(batch_size, features_len)
output = hitting_model(x)
print(output.shape)

torch.Size([1, 3])


## Defining the Loss

In [10]:
class EuclideanLoss(nn.Module):
    def __init__(self):
        super(EuclideanLoss, self).__init__()

    def forward(self, output, target):
        return torch.sqrt(torch.sum((output - target)**2))

In [11]:
criterion = EuclideanLoss()
optimizer = optim.Adam(hitting_model.parameters(), lr = 0.001)

## Loop

In [12]:
BALL_IS_THROWN = 0
BALL_HIT_THE_PADDLE = 1
BALL_HIT_THE_TABLE = 2

In [13]:
target_on_table = [0.2, 0.0] ## x, y position on the table where i want the ball to land

In [14]:
ball_init_position_x_range = [-2, -1]
ball_init_position_y_range = [-0.5, 0.5]
ball_init_position_z_range = [1.5, 2.5]

real_initial_velocity_x_range = [4, 8]
real_initial_velocity_y_range = [-0.5, 0.5]
real_initial_velocity_z_range = [-3, 1]

In [15]:
def generate_random_inputs():
    ball_init_position = [
        random.uniform(*ball_init_position_x_range),
        random.uniform(*ball_init_position_y_range),
        random.uniform(*ball_init_position_z_range)
    ]
    real_initial_velocity = [
        random.uniform(*real_initial_velocity_x_range),
        random.uniform(*real_initial_velocity_y_range),
        random.uniform(*real_initial_velocity_z_range)
    ]
    return ball_init_position, real_initial_velocity,

In [21]:
table_id = p.createCollisionShape(p.GEOM_BOX, halfExtents=[5, 5, 0.5])
def the_loop(ball_init_position = [-1.3, -0.2, 2.1], 
             real_initial_velocity = [7.0, -0.0, -2]):
    
    ##########################################################################################################################
    ################################################### Throw the ball #######################################################
    ##########################################################################################################################
    env.get_new_ball(position=ball_init_position)
    env.throw_ball(real_initial_velocity)
    state = BALL_IS_THROWN
    distance_ball_robot = p.getLinkState(env._robotic_arm, 0)[0][0] - ball_init_position[0]
    table_hit_point = None
    goal_position, estimated_init_vel = find_goal_position()## This goal position denotes the intercept point

    
    
    ##########################################################################################################################
    ####################################################### Hit the ball #####################################################
    ##########################################################################################################################
    if goal_position[1] > 1.5: ## If the z coordinate of the intercept point is below 1.5, the paddle will collide with the table
        moveit_plugin.plan_motion(x = -0.5,
                              y = goal_position[0],## This goal position denotes the intercept point
                              z = goal_position[1] - 1.4,
                              roll = 0,
                              pitch = 0,
                              yaw = np.pi/2)
        moveit_plugin.execute_plan() ## Reached the intercept point
        
        for i in range(500):
            if state == BALL_IS_THROWN and check_collision(env._ball, env._robotic_arm): ## Ensures that the ball hit the paddle
                state = BALL_HIT_THE_PADDLE
                print('ball has hit the paddle')  

                
                ### Now get the velocity prediction from the neural network
                features = [
                    ball_init_position[0], 
                    ball_init_position[1], 
                    ball_init_position[2],
                    estimated_init_vel[0], 
                    estimated_init_vel[1], 
                    estimated_init_vel[2],
                    distance_ball_robot,
                    goal_position[0], ## This goal position denotes the intercept point
                    goal_position[1],
                ]

                features = torch.tensor(features, dtype = torch.float32).unsqueeze(0)
                hitting_model.train() ## model in training mode
                optimizer.zero_grad()
                paddle_hitting_vel = hitting_model(features).squeeze().tolist()
                
                ## These 15 steps will be used to execute the hitting velocity
                for timer in range(15): 
                    p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                                            jointIndex=3,
                                            controlMode=p.VELOCITY_CONTROL,
                                            targetVelocity=paddle_hitting_vel[0],
                                           )
    
                    p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                                            jointIndex=4,
                                            controlMode=p.VELOCITY_CONTROL,
                                            targetVelocity=paddle_hitting_vel[1],
                                           )
                    p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                                            jointIndex=5,
                                            controlMode=p.VELOCITY_CONTROL,
                                            targetVelocity=paddle_hitting_vel[2],
                                           )
                    time.sleep(1/240.0)
                    p.stepSimulation()

                ## After these 15 time steps, we can make the velocity of the paddle 0
                p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                                            jointIndex=3,
                                            controlMode=p.VELOCITY_CONTROL,
                                            targetVelocity=0,
                                           )
                
                p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                                            jointIndex=4,
                                            controlMode=p.VELOCITY_CONTROL,
                                            targetVelocity=0,
                                           )
                p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                                            jointIndex=5,
                                            controlMode=p.VELOCITY_CONTROL,
                                            targetVelocity=0,
                                           )


    ##########################################################################################################################
    ############################################ Check if the ball hit the table #############################################
    ##########################################################################################################################
            if state == BALL_HIT_THE_PADDLE and check_collision(env._ball, env._table_body_id): # Ensures the sequence of events
                table_hit_point = env.get_state()['ball']['position'] ## Location on hte table where the ball hit it
                
                # Ensure that table_hit_point is not None
                table_hit_point_x, table_hit_point_y, table_hit_point_z = table_hit_point if table_hit_point else (None, None, None)
                if table_hit_point:
                    state = BALL_HIT_THE_TABLE
                    print('ball has hit the table at: ', table_hit_point)
                    landed_on_table_tensor = torch.tensor([table_hit_point_x, table_hit_point_y], dtype = torch.float32, requires_grad = True).unsqueeze(0)
                    target_on_table_tensor = torch.tensor(target_on_table, dtype = torch.float32, requires_grad = True).unsqueeze(0)
                    loss = criterion(landed_on_table_tensor, target_on_table_tensor)
                    loss.backward()
                    optimizer.step()
                    print(f'loss in this episode: {loss:.4f}, landed on this episode: ({table_hit_point_x:.2f}, {table_hit_point_y:.2f}), target was: ({target_on_table[0]:.2f}, {target_on_table[1]:.2f})')
            time.sleep(1/240.0)
            p.stepSimulation()
    return estimated_init_vel, goal_position, distance_ball_robot, table_hit_point

num_episodes = 10
for episode in range(num_episodes):
    print(f'Now Running Episode: {episode}')
    p.removeBody(env._table_body_id) # removing the old table
    table_visual_id = -1
    env._table_body_id = p.createMultiBody(baseMass=20000.0, baseCollisionShapeIndex=table_id, baseVisualShapeIndex=table_visual_id, basePosition=[0, 0, 0.7])
    p.changeDynamics(env._table_body_id, -1, restitution=0.8, lateralFriction = 0.5)  # Adjust the restitution as needed (e.g., 0.8 for a good bounce)
    p.removeBody(env._ball)
    ball_init_position, real_initial_velocity = generate_random_inputs()
    the_loop(ball_init_position=ball_init_position, 
             real_initial_velocity=real_initial_velocity)

Now Running Episode: {episode}
Now Running Episode: {episode}
Now Running Episode: {episode}
Now Running Episode: {episode}
Now Running Episode: {episode}
Now Running Episode: {episode}
Now Running Episode: {episode}
Now Running Episode: {episode}
Now Running Episode: {episode}
Now Running Episode: {episode}


In [19]:
torch.save(hitting_model.state_dict(), 'hitting_planner.pth')